---
title: "VRPCD with information exchange"
author: "FA"
date: 2021-01-07
tags: ["VRPCD", "realtime", "information exchange"]
categories: "research"
---

In [1]:
#*****************************************
# Necessary Package
#*****************************************
using Random
using JuMP
using LinearAlgebra
using GLPK

In [82]:
#*****************************************
# Network Property
#*****************************************
# P: set of pickup nodes
P = [2,3]

# D: set of delivery nodes
D = [6,7]

# O: cross-dock (assume single cross dock in the network)
O_pick = [4]
O_del = [5]

# O1: dummy cross-dock
O_dum_pick = [1]
O_dum_del = [8]

# pickup nodes
N_pickup = vcat(O_dum_pick, P, O_pick)

# delivery nodes
N_delivery = vcat(O_del, D, O_dum_del)

# all nodes
N = vcat(O_dum_pick, P, O_pick, O_del, D, O_dum_del)

# M: set of available vehicles
M = ["v1", "v2", "v3", "v4"]

# may be we need to position each entity in a graph to find distance/ time automatically
print(P,D,O_pick, O_del, N_pickup, N_delivery, N, M)


[2, 3][6, 7][4][5][1, 2, 3, 4][5, 6, 7, 8][1, 2, 3, 4, 5, 6, 7, 8]["v1", "v2", "v3", "v4"]

In [83]:
#****************************************
# parameters
#*****************************************
# p = number of pickup nodes
p = length(P)

# d = number of delivery nodes
d = length(D)

# o = number of cross dock
o_pick = length(O_pick)
o_del = length(O_del)


# p_cd: number of nodes in pickup process (dummy CD + P + CD)
p_cd = length(N_pickup)

# d_cd: number of nodes in delivery process (CD + P + dummy CD)
d_cd = length(N_delivery)

# n: total nodes in the network
n = length(N)

# number of availble vehicles
m = length(M)

# p_i: quantity of products to be collected at pickup node i; [include product type in future]
p_i = [0, 30, 50, 0, 0, 0, 0, 0]

# d_i: quantity of products to be delivered at delivery node i; [include product type in future]
d_i = [0, 0, 0, 0, 0, 30, 50, 0]

# capacity of vehicle (homogenous)
Q = 50

# Random.seed!(314)
# c_ij = the travel time (minutes)/cost between node i and node j; (assuming the cost is the same as the time req.)
t_ij= 
    [0 20 30 0 0 0 0 0;
    20 0 50 40 0 0 0 0;
    30 50 0 40 0 0 0 0;
    0 40 40 0 0 0 0 0;
    0 0 0 0 0 70 80 0;
    0 0 0 0 20 0 70 60;
    0 0 0 0 80 70 0 60;
    0 0 0 0 0 60 60 0]

# Cross-dock activity time
T_o = 15

# time horizon (16 hr a day)
T_max = 16*60

960

In [4]:
# ****************************************
# Create a JuMP model
# ****************************************
cd_modl = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [6]:
#*****************************************
# Constraints
#*****************************************

# Pick-up Process
#----------------


# [Pick-up Process] 01: only one vehicle can arrive at a pickup node
@constraint(cd_modl, p_node_entry[j= P],
                    sum(x_ijk[i,j,k] for i=vcat(O_dum_pick, P) for k=1:m if i != j) == 1)

#[Pick-up Process] 02: Only one vehicle depart from the pickup node
@constraint(cd_modl, p_node_exit[i= P],
                    sum(x_ijk[i,j,k] for j=vcat(P,O_pick) for k=1:m if i != j) == 1)

# [Pick-up Process] 03: Consecutive movement of vehicles
@constraint(cd_modl, cons_move_pick[l = P, k=1:m],
    sum(x_ijk[i,l,k] for i=vcat(O_dum_pick, P) if i != l) 
    -sum(x_ijk[l,j,k] for j=vcat(P,O_pick) if j != l) 
    == 0)

# [Pick-up Process] 04: vehicle leave at dummy cross dock
@constraint(cd_modl, p_dum_leave[i= O_dum_pick, k=1:m],
    sum(x_ijk[i,j,k] for j = P) <= 1)


# [Pick-up Process] 05: vehicle that leave at dummy cross dock visit cross dock immdiately after last pickup
@constraint(cd_modl, p_cd_entry[j = O_pick, k=1:m],
    sum(x_ijk[i,j,k] for i= P) <= 1)

# [Pick-up Process] 06: 
@constraint(cd_modl, p_pick_qnt[i = P, j = P,  k=1:m; i!=j],
    y_i[j] 
    - y_i[i] 
    - p_i[j] 
    + Q 
    - (Q*(x_ijk[i,j,k] + x_ijk[j,i,k])) 
    + ((p_i[j]+p_i[i])*x_ijk[j,i,k])
    >= 0)

# [Pick-up Process] 07: 
@constraint(cd_modl, p_pick_qnt2[i = O_dum_pick, j = P, k=1:m],
    y_i[j]
    - x_ijk[i,j,k]*p_i[j]
    - (1-x_ijk[i,j,k])*Q
    <= 0)

# [Pick-up Process] 08a: 
@constraint(cd_modl, pick_cap1[i = P],
    y_i[i] >= p_i[i] )

# [Pick-up Process] 08b: 
@constraint(cd_modl, pick_cap2[i = P],
    y_i[i] <= Q)

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, [2, 3]
And data, a 2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 pick_cap2[2] : y_i[2] <= 50.0
 pick_cap2[3] : y_i[3] <= 50.0

In [7]:
#*****************************************
# Constraints
#*****************************************

# Delivery Process
#----------------

# [Delivery Process] 01: only one vehicle can arrive at a pickup node
@constraint(cd_modl, d_node_entry[j= D],
                    sum(x_ijk[i,j,k] for i=vcat(O_del, D) for k=1:m if i != j) == 1)

# [Delivery Process] 02: Only one vehicle depart from the pickup node
@constraint(cd_modl, d_node_exit[i= D],
                    sum(x_ijk[i,j,k] for j=vcat(D,O_dum_del) for k=1:m if i != j) == 1)

# [Delivery Process] 03: Consecutive movement of vehicles
@constraint(cd_modl, cons_move_del[l = D, k=1:m],
    sum(x_ijk[i,l,k] for i=vcat(O_del, D) if i != l) 
    -sum(x_ijk[l,j,k] for j=vcat(D,O_dum_del) if j != l) 
    == 0)

# [Delivery Process] 04: vehicle enter at dummy cross dock after last delivery
@constraint(cd_modl, d_dum_enter[k=1:m],
    sum(x_ijk[i,j,k] for i= D for j = O_dum_del) <= 1)

# [Delivery Process] 05: vehicle leave cross dock
@constraint(cd_modl, d_cd_leave[k=1:m],
    sum(x_ijk[i,j,k] for i= O_del for j = D) <= 1)

# [Delivery Process] 06: 
@constraint(cd_modl, d_del_qnt[k=1:m, i = D, j = D; i!=j],
    y_i[j] 
    - y_i[i] 
    - d_i[j] 
    + Q 
    - (Q*(x_ijk[i,j,k] + x_ijk[j,i,k])) 
    + ((d_i[j]+d_i[i])*x_ijk[j,i,k])
    >= 0)

# [Delivery Process] 07: 
@constraint(cd_modl, d_del_qnt2[i = O_del, j = D, k=1:m],
    y_i[j]
    - x_ijk[i,j,k]*d_i[j]
    - (1-x_ijk[i,j,k])*Q
    <= 0)

# [Delivery Process] 08a: 
@constraint(cd_modl, del_cap1[i = D],
    y_i[i] >= d_i[i] )

# [Delivery Process] 08b: 
@constraint(cd_modl, del_cap2[i = D],
    y_i[i] <= Q)

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, [6, 7]
And data, a 2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 del_cap2[6] : y_i[6] <= 50.0
 del_cap2[7] : y_i[7] <= 50.0

In [8]:
#*****************************************
# Constraints
#*****************************************

# @cross dock:: Connection Between Pickup and Delivery Process (using time)
#------------------------------------------------------------

# [cross-dock] 01: track time for product collection in the pickup process
@constraint(cd_modl, cd_prod_col[k = 1: m],
    AT_k[k]
    -sum(t_ij[i,j] * x_ijk[i,j,k] for i=vcat(O_dum_pick, P) for j=vcat(P, O_pick) if i != j) >= 0)

# [cross-dock] 02: all trucks arrive at the same time at cross-dock
@constraint(cd_modl, cd_truck_arr[k = 1: m, k_prime = 1:m; k != k_prime],
    AT_k[k] - AT_k[k_prime] == 0)

# [cross-dock] 03: total time at the pickup process
@constraint(cd_modl, time_pick[k = 1: m],
    t_pick[o_pick] - AT_k[k] >= 0)

# [cross-dock] 04: total transportation time and processing time does not exceed the planning horizon T_max
@constraint(cd_modl, time_hor[k = 1: m],
    t_pick[o_pick]+ T_o + sum(t_ij[i,j] * x_ijk[i,j,k] for i = vcat(O_del, D) for j = vcat(D, O_dum_del)) - T_max <= 0)


4-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 time_hor[1] : 70 x_ijk[5,6,1] + 70 x_ijk[7,6,1] + 80 x_ijk[5,7,1] + 70 x_ijk[6,7,1] + 60 x_ijk[6,8,1] + 60 x_ijk[7,8,1] + t_pick[1] <= 945.0
 time_hor[2] : 70 x_ijk[5,6,2] + 70 x_ijk[7,6,2] + 80 x_ijk[5,7,2] + 70 x_ijk[6,7,2] + 60 x_ijk[6,8,2] + 60 x_ijk[7,8,2] + t_pick[1] <= 945.0
 time_hor[3] : 70 x_ijk[5,6,3] + 70 x_ijk[7,6,3] + 80 x_ijk[5,7,3] + 70 x_ijk[6,7,3] + 60 x_ijk[6,8,3] + 60 x_ijk[7,8,3] + t_pick[1] <= 945.0
 time_hor[4] : 70 x_ijk[5,6,4] + 70 x_ijk[7,6,4] + 80 x_ijk[5,7,4] + 70 x_ijk[6,7,4] + 60 x_ijk[6,8,4] + 60 x_ijk[7,8,4] + t_pick[1] <= 945.0

In [9]:
#*****************************************
# Objective
#*****************************************

@objective(cd_modl, Min, sum(t_ij[i,j]*x_ijk[i,j,k] for i=vcat(O_dum_pick, P) for j=vcat(P, O_pick) for k=1:m) + sum(t_ij[i,j]*x_ijk[i,j,k] for i=vcat(O_del, D) for j=vcat(D, O_dum_del) for k=1:m))

20 x_ijk[1,2,1] + 20 x_ijk[1,2,2] + 20 x_ijk[1,2,3] + 20 x_ijk[1,2,4] + 30 x_ijk[1,3,1] + 30 x_ijk[1,3,2] + 30 x_ijk[1,3,3] + 30 x_ijk[1,3,4] + 50 x_ijk[2,3,1] + 50 x_ijk[2,3,2] + 50 x_ijk[2,3,3] + 50 x_ijk[2,3,4] + 40 x_ijk[2,4,1] + 40 x_ijk[2,4,2] + 40 x_ijk[2,4,3] + 40 x_ijk[2,4,4] + 50 x_ijk[3,2,1] + 50 x_ijk[3,2,2] + 50 x_ijk[3,2,3] + 50 x_ijk[3,2,4] + 40 x_ijk[3,4,1] + 40 x_ijk[3,4,2] + 40 x_ijk[3,4,3] + 40 x_ijk[3,4,4] + 70 x_ijk[5,6,1] + 70 x_ijk[5,6,2] + 70 x_ijk[5,6,3] + 70 x_ijk[5,6,4] + 80 x_ijk[5,7,1] + 80 x_ijk[5,7,2] + 80 x_ijk[5,7,3] + 80 x_ijk[5,7,4] + 70 x_ijk[6,7,1] + 70 x_ijk[6,7,2] + 70 x_ijk[6,7,3] + 70 x_ijk[6,7,4] + 60 x_ijk[6,8,1] + 60 x_ijk[6,8,2] + 60 x_ijk[6,8,3] + 60 x_ijk[6,8,4] + 70 x_ijk[7,6,1] + 70 x_ijk[7,6,2] + 70 x_ijk[7,6,3] + 70 x_ijk[7,6,4] + 60 x_ijk[7,8,1] + 60 x_ijk[7,8,2] + 60 x_ijk[7,8,3] + 60 x_ijk[7,8,4]

In [ ]:
print(cd_modl)

In [11]:
optimize!(cd_modl)

In [12]:
@show objective_value(cd_modl)

objective_value(cd_modl) = 400.0


400.0

In [13]:
@show value.(x_ijk);

value.(x_ijk) = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]

[0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]

[0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 

In [14]:
@show value.(y_i);
prod_qnt = value.(y_i)

value.(y_i) = [0.0, 30.0, 50.0, 0.0, 0.0, 30.0, 50.0, 0.0]


8-element Array{Float64,1}:
  0.0
 30.0
 50.0
  0.0
  0.0
 30.0
 50.0
  0.0

In [15]:
@show value.(AT_k);

value.(AT_k) = [70.0, 70.0, 70.0, 70.0]


In [16]:
@show value.(t_pick);

value.(t_pick) = [70.0]


In [17]:
# vehicle routing information

for k=1:m
    print("\nveh: ", k, "\t")
    for i= 1:n
        for j=1:n
            if value.(x_ijk[i,j,k]) == 1
                print(i, "-->", j, "\t")
            end
        end
    end
end


veh: 1	5-->7	7-->8	
veh: 2	1-->3	3-->4	5-->6	6-->8	
veh: 3	
veh: 4	1-->2	2-->4	

In [18]:
# vehicle estimated arrival time at cross dock

for k=1:m
    print("\nveh: ", k, "\t")
    temp = 0
    for i=1:n
        for j=1:n
            if value.(x_ijk[i,j,k]) == 1
                temp = temp + t_ij[i,j]
                if j == 4 || j==8
                    print("est. arr. time @ CD: ", temp)
                end
            end
        end
    end
end


veh: 1	est. arr. time @ CD: 140
veh: 2	est. arr. time @ CD: 70est. arr. time @ CD: 200
veh: 3	
veh: 4	est. arr. time @ CD: 60

In [19]:
using DataFrames
pick_amt = DataFrame(node =[], amount =[])

for k in keys(y_i)
    val = value.(y_i[k...])
    push!(pick_amt, vcat(k..., val))
end

pick_amt


,node,amount
,Any,Any
1,1.0,0.0
2,2.0,30.0
3,3.0,50.0
4,4.0,0.0
5,5.0,0.0
6,6.0,30.0
7,7.0,50.0
8,8.0,0.0


In [39]:
mvmt_pick = DataFrame(from=[], to=[], veh=[], status=[])
mvmt_del = DataFrame(from=[], to=[], veh=[], status=[])

for k in keys(x_ijk)
    val = getvalue.(x_ijk[k])
    #print([Tuple(k)...][2]) # vehicles end node of a movement
    end_node = [Tuple(k)...][2]
    
    # record the routing of pickup process
    if val > 0 && end_node <= 4
        push!(mvmt_pick, vcat([Tuple(k)...], val))
    end
    
    # record the routing of delivery process
    if val > 0 && end_node > 4
        push!(mvmt_del, vcat([Tuple(k)...], val))
    end
end

In [40]:
mvmt_pick

,from,to,veh,status
,Any,Any,Any,Any
1,1.0,3.0,2.0,1.0
2,3.0,4.0,2.0,1.0
3,1.0,2.0,4.0,1.0
4,2.0,4.0,4.0,1.0


In [41]:
mvmt_del

,from,to,veh,status
,Any,Any,Any,Any
1,5.0,7.0,1.0,1.0
2,7.0,8.0,1.0,1.0
3,5.0,6.0,2.0,1.0
4,6.0,8.0,2.0,1.0


In [42]:
# inialize empty array to capture travel time, tt
tt_pick = []
prod_pick = []
for i=1:nrow(mvmt_pick)
    x = Int(mvmt_pick[i,1])
    y = Int(mvmt_pick[i,2])
    prod_amt = pick_amt[y, 2]
    print(x, "\t", y, "\t", prod_amt, "\n")
    append!(tt_pick,t_ij[x,y]) ## append data to the tt array
    append!(prod_pick,prod_amt)
end

tt_pick
prod_pick
insert!(mvmt_pick, 5, tt_pick, :tt)
insert!(mvmt_pick, 6, prod_pick, :prod_amt)

1	3	50.0
3	4	0.0
1	2	30.0
2	4	0.0


,from,to,veh,status,tt,prod_amt
,Any,Any,Any,Any,Any,Any
1,1.0,3.0,2.0,1.0,30,50.0
2,3.0,4.0,2.0,1.0,40,0.0
3,1.0,2.0,4.0,1.0,20,30.0
4,2.0,4.0,4.0,1.0,40,0.0


In [137]:
# inialize empty array to capture travel time, tt
tt_del = []
prod_del = []
for i=1:nrow(mvmt_del)
    x = Int(mvmt_del[i,1])
    y = Int(mvmt_del[i,2])
    prod_amt = pick_amt[y, 2]
    append!(tt_del,t_ij[x,y]) ## append data to the tt array
    append!(prod_del,prod_amt)
end

tt_del

#insert!(mvmt_del, 5, tt_del, :tt)
insert!(mvmt_del, 6, prod_del, :prod_del)

,from,to,veh,status,tt,prod_del
,Any,Any,Any,Any,Any,Any
1,5.0,7.0,1.0,1.0,80,50.0
2,7.0,8.0,1.0,1.0,60,0.0
3,5.0,6.0,2.0,1.0,70,30.0
4,6.0,8.0,2.0,1.0,60,0.0


In [25]:
# group by vehicle number and sum the travel time to get the estimated arrival at cross-dock
cd_arr_time = combine(groupby(mvmt_pick,  [:veh]) , :tt => sum)

,veh,tt_sum
,Any,Int64
1,2.0,70
2,4.0,60


In [44]:
# group by vehicle number and sum the travel time to get the estimated arrival at cross-dock
cd_product = combine(groupby(mvmt_pick,  [:veh]) , :prod_amt => sum)

,veh,prod_amt_sum
,Any,Float64
1,2.0,50.0
2,4.0,30.0


In [27]:
join(cd_arr_time, cd_product, on = :veh)

,veh,tt_sum,prod_amt_sum
,Any,Int64,Float64
1,2.0,70,50.0
2,4.0,60,30.0


In [61]:
del_time = combine(groupby(mvmt_del,  [:veh]) , :tt => sum)

est_del = []
for i= 1:nrow(del_time)
    temp = del_time[i, 2] + value.(t_pick)[1]  # add the latest time for pickup process
    append!(est_del, temp)
end

insert!(del_time, 3, est_del, :est_del)

Any[210.0, 200.0]

,veh,tt_sum,est_del
,Any,Int64,Any
1,1.0,140,210.0
2,2.0,130,200.0


In [68]:
for t in [0:15:120;]
    print("\n\ncurrent time: ", t)
    # check status of each vehicle
    
    # where is the position of each vehicle?
    # is the current time less than t_pick? if yes "pickup process"; "delivery process" other wise
    tt = []
    tt_del = []
    if t <= value.(t_pick)[1]
        print("\nvehicles in PICKUP process")
        for i=1:nrow(cd_arr_time)
            curr_arr_time = cd_arr_time[i,2]
            update_time = curr_arr_time - t
            append!(tt, update_time)
        end
        
        # update time for delivery process
        for i=1:nrow(del_time)
            curr_del_time = del_time[i,3]
            update_time = curr_del_time - t
            append!(tt_del, update_time)
        end
        
        # print table
        cd_arr_time.est = tt
        del_time.est = tt_del
    else
        print("\nvehicles in DELIVERY process")
    end
    print("\n Pickup Process", cd_arr_time, "\n")
    print("\n Delivery Process:", del_time, "\n")

end



current time: 0
vehicles in PICKUP process
 Pickup Process2×3 DataFrame
│ Row │ veh │ tt_sum │ est │
│     │ Any │ Int64  │ Any │
├─────┼─────┼────────┼─────┤
│ 1   │ 2.0 │ 70     │ 70  │
│ 2   │ 4.0 │ 60     │ 60  │

 Delivery Process:2×4 DataFrame
│ Row │ veh │ tt_sum │ est_del │ est   │
│     │ Any │ Int64  │ Any     │ Any   │
├─────┼─────┼────────┼─────────┼───────┤
│ 1   │ 1.0 │ 140    │ 210.0   │ 210.0 │
│ 2   │ 2.0 │ 130    │ 200.0   │ 200.0 │


current time: 15
vehicles in PICKUP process
 Pickup Process2×3 DataFrame
│ Row │ veh │ tt_sum │ est │
│     │ Any │ Int64  │ Any │
├─────┼─────┼────────┼─────┤
│ 1   │ 2.0 │ 70     │ 55  │
│ 2   │ 4.0 │ 60     │ 45  │

 Delivery Process:2×4 DataFrame
│ Row │ veh │ tt_sum │ est_del │ est   │
│     │ Any │ Int64  │ Any     │ Any   │
├─────┼─────┼────────┼─────────┼───────┤
│ 1   │ 1.0 │ 140    │ 210.0   │ 195.0 │
│ 2   │ 2.0 │ 130    │ 200.0   │ 185.0 │


current time: 30
vehicles in PICKUP process
 Pickup Process2×3 DataFrame
│ Row │ ve

In [29]:
# vehicle status: first digit indicate whether it is used in none(0)/pickup(1)/delivery(2)/both(3); last two digits indicate active(10)/broke(99)/available(00)
veh_status = [10, 99, 10, 10]
veh_use = [1,3,0,2]

4-element Array{Int64,1}:
 1
 3
 0
 2

In [62]:
# current time
t = 15

veh_break = findall( x -> x == 999, veh_status)
num_veh_break = length(veh_break)

veh_avl = 
num_veh_avl = length(m) - num_veh_break

3

In [158]:
# updated available veh

m2 = [1,3, 4]

# ****************************************
# Create a JuMP model
# ****************************************
cd_modl2 = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [159]:
# ****************************************
# Decision variables
#*****************************************
# x_ijk = 1 if vehicle k travels from node i to node j; 0 otherwise;
@variable(cd_modl2, p_ijk[i=1:n, j=1:n, k=1:m], Bin)

# y_i = total quantity of collected/delivered products after leaving node i
@variable(cd_modl2, y_i[i = 1:n], Int)




8-element Array{VariableRef,1}:
 y_i[1]
 y_i[2]
 y_i[3]
 y_i[4]
 y_i[5]
 y_i[6]
 y_i[7]
 y_i[8]

In [160]:
#*****************************************
# Constraints
#*****************************************

# Delivery Process
#----------------

# [Delivery Process] 01: only one vehicle can arrive at a pickup node
@constraint(cd_modl2, d_node_entry[j= D],
                    sum(p_ijk[i,j,k] for i=vcat(O_del, D) for k=m2 if i != j) == 1)

# [Delivery Process] 02: Only one vehicle depart from the pickup node
@constraint(cd_modl2, d_node_exit[i= D],
                    sum(p_ijk[i,j,k] for j=vcat(D,O_dum_del) for k=m2 if i != j) == 1)

# [Delivery Process] 03: Consecutive movement of vehicles
@constraint(cd_modl2, cons_move_del[l = D, k=m2],
    sum(p_ijk[i,l,k] for i=vcat(O_del, D) if i != l) 
    -sum(p_ijk[l,j,k] for j=vcat(D,O_dum_del) if j != l) 
    == 0)

# [Delivery Process] 04: vehicle enter at dummy cross dock after last delivery
@constraint(cd_modl2, d_dum_enter[k=m2],
    sum(p_ijk[i,j,k] for i= D for j = O_dum_del) <= 1)

# [Delivery Process] 05: vehicle leave cross dock
@constraint(cd_modl2, d_cd_leave[k=m2],
    sum(p_ijk[i,j,k] for i= O_del for j = D) <= 1)

# [Delivery Process] 06: 
@constraint(cd_modl2, d_del_qnt[k=m, i = D, j = D; i!=j],
    y_i[j] 
    - y_i[i] 
    - d_i[j] 
    + Q 
    - (Q*(p_ijk[i,j,k] + p_ijk[j,i,k])) 
    + ((d_i[j]+d_i[i])*p_ijk[j,i,k])
    >= 0)

# [Delivery Process] 07: 
@constraint(cd_modl2, d_del_qnt2[i = O_del, j = D, k=m2],
    y_i[j]
    - p_ijk[i,j,k]*d_i[j]
    - (1-p_ijk[i,j,k])*Q
    <= 0)

3-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},3,...} with index sets:
    Dimension 1, [5]
    Dimension 2, [6, 7]
    Dimension 3, [1, 3, 4]
And data, a 1×2×3 Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},3}:
[:, :, 1] =
 d_del_qnt2[5,6,1] : 20 p_ijk[5,6,1] + y_i[6] <= 50.0  …  d_del_qnt2[5,7,1] : y_i[7] <= 50.0

[:, :, 3] =
 d_del_qnt2[5,6,3] : 20 p_ijk[5,6,3] + y_i[6] <= 50.0  …  d_del_qnt2[5,7,3] : y_i[7] <= 50.0

[:, :, 4] =
 d_del_qnt2[5,6,4] : 20 p_ijk[5,6,4] + y_i[6] <= 50.0  …  d_del_qnt2[5,7,4] : y_i[7] <= 50.0

In [161]:
#*****************************************
# Objective
#*****************************************

@objective(cd_modl2, Min, sum(t_ij[i,j]*p_ijk[i,j,k] for i=vcat(O_del, D) for j=vcat(D, O_dum_del) for k=m2))

70 p_ijk[5,6,1] + 70 p_ijk[5,6,3] + 70 p_ijk[5,6,4] + 80 p_ijk[5,7,1] + 80 p_ijk[5,7,3] + 80 p_ijk[5,7,4] + 70 p_ijk[6,7,1] + 70 p_ijk[6,7,3] + 70 p_ijk[6,7,4] + 60 p_ijk[6,8,1] + 60 p_ijk[6,8,3] + 60 p_ijk[6,8,4] + 70 p_ijk[7,6,1] + 70 p_ijk[7,6,3] + 70 p_ijk[7,6,4] + 60 p_ijk[7,8,1] + 60 p_ijk[7,8,3] + 60 p_ijk[7,8,4]

In [162]:
optimize!(cd_modl2)

In [163]:
# vehicle routing information

for k=m
    print("\nveh: ", k, "\t")
    for i= 1:n
        for j=1:n
            if value.(p_ijk[i,j,k]) == 1
                print(i, "-->", j, "\t")
            end
        end
    end
end


veh: 4	

In [149]:
@show value.(p_ijk)

value.(p_ijk) = 3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, Base.OneTo(8)
    Dimension 2, Base.OneTo(8)
    Dimension 3, [1, 3, 4]
And data, a 8×8×3 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0


3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, Base.OneTo(8)
    Dimension 2, Base.OneTo(8)
    Dimension 3, [1, 3, 4]
And data, a 8×8×3 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  

In [133]:
# ****************************************
# Create a JuMP model
# ****************************************
cd_modl3 = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [134]:
# ****************************************
# Decision variables
#*****************************************
N = [1, 3, 4, 5, 6, 7, 8]
m3 = [3]

# x_ijk = 1 if vehicle k travels from node i to node j; 0 otherwise;
@variable(cd_modl3, x3_ijk[i=N, j=N, k=m3], Bin)

# y_i = total quantity of collected/delivered products after leaving node i
@variable(cd_modl3, y3_i[i = N], Int)

# t_pickup = latest arrival time a cross dock
@variable(cd_modl3, t3_pick[pick = 1:o_pick])

# AT_k = arrival time of vehicle k at the cross-dock
@variable(cd_modl3, AT3_k[k =m3])

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [3]
And data, a 1-element Array{VariableRef,1}:
 AT3_k[3]

In [135]:
#*****************************************
# Constraints
#*****************************************

# Pick-up Process
#----------------
P3 = [3]

# [Pick-up Process] 01: only one vehicle can arrive at a pickup node
@constraint(cd_modl3, p_node_entry[j= P3],
                    sum(x3_ijk[i,j,k] for i=vcat(O_dum_pick, P3) for k=m3 if i != j) == 1)

#[Pick-up Process] 02: Only one vehicle depart from the pickup node
@constraint(cd_modl3, p_node_exit[i= P3],
                    sum(x3_ijk[i,j,k] for j=vcat(P3,O_pick) for k=m3 if i != j) == 1)

# [Pick-up Process] 03: Consecutive movement of vehicles
@constraint(cd_modl3, cons_move_pick[l = P3, k=m3],
    sum(x3_ijk[i,l,k] for i=vcat(O_dum_pick, P3) if i != l) 
    -sum(x3_ijk[l,j,k] for j=vcat(P3,O_pick) if j != l) 
    == 0)

# [Pick-up Process] 04: vehicle leave at dummy cross dock
@constraint(cd_modl3, p_dum_leave[i= O_dum_pick, k=m3],
    sum(x3_ijk[i,j,k] for j = P3) <= 1)


# [Pick-up Process] 05: vehicle that leave at dummy cross dock visit cross dock immdiately after last pickup
@constraint(cd_modl3, p_cd_entry[j = O_pick, k=m3],
    sum(x3_ijk[i,j,k] for i= P3) <= 1)

# [Pick-up Process] 06: 
@constraint(cd_modl3, p_pick_qnt[i = P3, j = P3,  k=m3; i!=j],
    y3_i[j] 
    - y3_i[i] 
    - p_i[j] 
    + Q 
    - (Q*(x3_ijk[i,j,k] + x3_ijk[j,i,k])) 
    + ((p_i[j]+p_i[i])*x3_ijk[j,i,k])
    >= 0)

# [Pick-up Process] 07: 
@constraint(cd_modl3, p_pick_qnt2[i = O_dum_pick, j = P3, k=m3],
    y3_i[j]
    - x3_ijk[i,j,k]*p_i[j]
    - (1-x3_ijk[i,j,k])*Q
    <= 0)

# [Pick-up Process] 08a: 
@constraint(cd_modl3, pick_cap1[i = P3],
    y3_i[i] >= p_i[i] )

# [Pick-up Process] 08b: 
@constraint(cd_modl3, pick_cap2[i = P3],
    y3_i[i] <= Q)

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, [3]
And data, a 1-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 pick_cap2[3] : y3_i[3] <= 50.0

In [136]:
#*****************************************
# Constraints
#*****************************************
m3_del  = [1, 3, 4]

# Delivery Process
#----------------

# [Delivery Process] 01: only one vehicle can arrive at a pickup node
@constraint(cd_modl3, d_node_entry[j= D],
                    sum(x3_ijk[i,j,k] for i=vcat(O_del, D) for k=m3_del if i != j) == 1)

# [Delivery Process] 02: Only one vehicle depart from the pickup node
@constraint(cd_modl3, d_node_exit[i= D],
                    sum(x3_ijk[i,j,k] for j=vcat(D,O_dum_del) for k=m3_del if i != j) == 1)

# [Delivery Process] 03: Consecutive movement of vehicles
@constraint(cd_modl3, cons_move_del[l = D, k=m3_del],
    sum(x3_ijk[i,l,k] for i=vcat(O_del, D) if i != l) 
    -sum(x3_ijk[l,j,k] for j=vcat(D,O_dum_del) if j != l) 
    == 0)

# [Delivery Process] 04: vehicle enter at dummy cross dock after last delivery
@constraint(cd_modl3, d_dum_enter[k=m3_del],
    sum(x3_ijk[i,j,k] for i= D for j = O_dum_del) <= 1)

# [Delivery Process] 05: vehicle leave cross dock
@constraint(cd_modl3, d_cd_leave[k=m3_del],
    sum(x3_ijk[i,j,k] for i= O_del for j = D) <= 1)

# [Delivery Process] 06: 
@constraint(cd_modl3, d_del_qnt[i = D, j = D, k=m3_del; i!=j],
    y3_i[j] 
    - y3_i[i] 
    - d_i[j] 
    + Q 
    - (Q*(x3_ijk[i,j,k] + x3_ijk[j,i,k])) 
    + ((d_i[j]+d_i[i])*x3_ijk[j,i,k])
    >= 0)

# [Delivery Process] 07: 
@constraint(cd_modl3, d_del_qnt2[i = O_del, j = D, k=m3_del],
    y3_i[j]
    - x3_ijk[i,j,k]*d_i[j]
    - (1-x3_ijk[i,j,k])*Q
    <= 0)

LoadError: KeyError: key 1 not found

In [122]:
#*****************************************
# Constraints
#*****************************************

# @cross dock:: Connection Between Pickup and Delivery Process (using time)
#------------------------------------------------------------

# [cross-dock] 01: track time for product collection in the pickup process
@constraint(cd_modl3, cd_prod_col[k = 1: m],
    AT3_k[k]
    -sum(t_ij[i,j] * x3_ijk[i,j,k] for i=vcat(O_dum_pick, P3) for j=vcat(P3, O_pick) if i != j) >= 0)

# [cross-dock] 02: all trucks arrive at the same time at cross-dock
@constraint(cd_modl3, cd_truck_arr[k = m3, k_prime = m3; k != k_prime],
    AT3_k[k] - AT3_k[k_prime] == 0)

# [cross-dock] 03: total time at the pickup process
@constraint(cd_modl3, time_pick[k = m3],
    t3_pick[o_pick] - AT3_k[k] >= 0)

# [cross-dock] 04: total transportation time and processing time does not exceed the planning horizon T_max
@constraint(cd_modl3, time_hor[k = 1: m],
    t3_pick[o_pick]+ T_o + sum(t_ij[i,j] * x3_ijk[i,j,k] for i = vcat(O_del, D) for j = vcat(D, O_dum_del)) - T_max <= 0)

4-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 time_hor[1] : 70 x3_ijk[5,6,1] + 70 x3_ijk[7,6,1] + 80 x3_ijk[5,7,1] + 70 x3_ijk[6,7,1] + 60 x3_ijk[6,8,1] + 60 x3_ijk[7,8,1] + t3_pick[1] <= 945.0
 time_hor[2] : 70 x3_ijk[5,6,2] + 70 x3_ijk[7,6,2] + 80 x3_ijk[5,7,2] + 70 x3_ijk[6,7,2] + 60 x3_ijk[6,8,2] + 60 x3_ijk[7,8,2] + t3_pick[1] <= 945.0
 time_hor[3] : 70 x3_ijk[5,6,3] + 70 x3_ijk[7,6,3] + 80 x3_ijk[5,7,3] + 70 x3_ijk[6,7,3] + 60 x3_ijk[6,8,3] + 60 x3_ijk[7,8,3] + t3_pick[1] <= 945.0
 time_hor[4] : 70 x3_ijk[5,6,4] + 70 x3_ijk[7,6,4] + 80 x3_ijk[5,7,4] + 70 x3_ijk[6,7,4] + 60 x3_ijk[6,8,4] + 60 x3_ijk[7,8,4] + t3_pick[1] <= 945.0

In [124]:
#*****************************************
# Objective
#*****************************************

@objective(cd_modl3, Min, 
    sum(t_ij[i,j]*x3_ijk[i,j,k] for i=vcat(O_dum_pick, P3) for j=vcat(P3, O_pick) for k=m3) 
    + sum(t_ij[i,j]*x3_ijk[i,j,k] for i=vcat(O_del, D) for j=vcat(D, O_dum_del) for k=m3_del))

30 x3_ijk[1,3,3] + 40 x3_ijk[3,4,3] + 70 x3_ijk[5,6,1] + 70 x3_ijk[5,6,3] + 70 x3_ijk[5,6,4] + 80 x3_ijk[5,7,1] + 80 x3_ijk[5,7,3] + 80 x3_ijk[5,7,4] + 70 x3_ijk[6,7,1] + 70 x3_ijk[6,7,3] + 70 x3_ijk[6,7,4] + 60 x3_ijk[6,8,1] + 60 x3_ijk[6,8,3] + 60 x3_ijk[6,8,4] + 70 x3_ijk[7,6,1] + 70 x3_ijk[7,6,3] + 70 x3_ijk[7,6,4] + 60 x3_ijk[7,8,1] + 60 x3_ijk[7,8,3] + 60 x3_ijk[7,8,4]

In [165]:
optimize!(cd_modl3)

In [166]:
@show objective_value(cd_modl3)

objective_value(cd_modl3) = 0.0


0.0

In [128]:
# vehicle routing information

for k=m3_del
    print("\nveh: ", k, "\t")
    for i= 1:n
        for j=1:n
            if value.(x3_ijk[i,j,k]) == 1
                print(i, "-->", j, "\t")
            end
        end
    end
end


veh: 1	5-->6	6-->8	
veh: 3	1-->3	2-->3	3-->4	5-->7	7-->8	
veh: 4	

In [129]:
# vehicle routing information

for k=m3
    print("\nveh: ", k, "\t")
    for i= 1:n
        for j=1:n
            if value.(x3_ijk[i,j,k]) == 1
                print(i, "-->", j, "\t")
            end
        end
    end
end


veh: 3	1-->3	2-->3	3-->4	5-->7	7-->8	